In [201]:
from ultralytics import YOLO
import numpy as np
import glob
from tqdm import tqdm
from ultralytics.data.augment import Albumentations


In [202]:
def __init__(self, p=1.0):
    """Initialize the transform object for YOLO bbox formatted params."""
    self.p = p
    self.transform = None
    prefix = colorstr("albumentations: ")
    try:
        import albumentations as A

        # check_version(A.__version__, "1.0.3", hard=True)  # version requirement

        # Transforms
        T = [
            A.Rotate(limit = 10, p=0.0),
            A.Blur(p=0.0),
            A.MedianBlur(p=0.0),
            A.ToGray(p=0.0),
            A.CLAHE(p=0.0),
            # A.RandomBrightnessContrast(brightness_limit=(-0.25, -0.25), contrast_limit=(-0.25, -0.25), p=0),
            A.RandomGamma(p=0.0),
            A.ImageCompression(quality_lower=75, p=0.0),

        ]
        self.contains_spatial = any(transform.__class__.__name__ in spatial_transforms for transform in T)
        self.transform = (
            A.Compose(T, bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]))
            if self.contains_spatial
            else A.Compose(T)
        )

        LOGGER.info(prefix + ", ".join(f"{x}".replace("always_apply=False, ", "") for x in T if x.p))
    except ImportError:  # package not installed, skip
        pass
    except Exception as e:
        LOGGER.info(f"{prefix}{e}")


Albumentations.__init__ = __init__


In [209]:
# inference(test) code
# train26 : goat

# test -> re-bbox -> completed (urin_by_class)

'''
model = YOLO("runs/detect/train26/weights/best.pt", verbose=False)

# data_path = "/home/jupyter-heypaprika/datasets/urin_check_dataset/Z-FLIP_unlabelled/"
data_path = "/home/jupyter-heypaprika/input/urin_by_class/*/*/*"
files = glob.glob(data_path)
total = 0
total_wrong = 0
for file in tqdm(files):
    if not "jpg" in file:
        continue
    results = model(file, imgsz=224, verbose=False)
    boxes = results[0].boxes
    cur_cls = map(int, boxes.cls.tolist())
    cur_xywhn = boxes.xywhn.cpu().numpy()
    cur_xywhn = np.array([["{:.6f}".format(x) for x in row] for row in cur_xywhn])
    cur_list = []
    for cls, xywhn in zip(cur_cls, cur_xywhn):
        xywhn = np.insert(xywhn, 0, str(cls))
        cur_list.append(xywhn)
    
    text_data = "\n".join([" ".join(row) for row in cur_list])
    
    # Save to a .txt file
    file_path = file.replace(file[-4:], ".txt")
    with open(file_path, "w") as file:
        file.write(text_data)
    predict_value = 60
    if len(cur_list) <= predict_value:
        len_boxes = len(cur_list)
        wrong = predict_value - len_boxes
        # print("total:", len_boxes, ", wrong:", wrong)
        total += predict_value
        total_wrong += wrong

avg_wrong = total_wrong / total
print(avg_wrong)


# results[0].show()
'''


100%|██████████| 2363/2363 [01:57<00:00, 20.04it/s]

0.0017444981213097155


In [176]:
data_path = "/home/jupyter-heypaprika/datasets/urin_check_dataset/Z-FLIP_unlabelled/"
files = glob.glob(data_path+"*.jpg")
file = files[0]
print(file)
results = model(file, imgsz=224, verbose=False)
boxes = results[0].boxes
cur_cls = list(map(int, boxes.cls.tolist()))
cur_xywhn = boxes.xywhn.cpu().numpy()
cur_xywhn = np.array([["{:.6f}".format(x) for x in row] for row in cur_xywhn])
cur_list = []
print(cur_cls)
print(boxes)
# for cls, xywhn in zip(cur_cls, cur_xywhn):
print(results[0].orig_img.shape)

/home/jupyter-heypaprika/datasets/urin_check_dataset/Z-FLIP_unlabelled/20241015_165447.jpg
[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0')
conf: tensor([0.9074, 0.9056, 0.9033, 0.8948, 0.8940, 0.8902, 0.8865, 0.8837, 0.8824, 0.8823, 0.8816, 0.8772, 0.8729, 0.8723, 0.8720, 0.8718, 0.8705, 0.8702, 0.8690, 0.8667, 0.8652, 0.8645, 0.8644, 0.8640, 0.8626, 0.8620, 0.8617, 0.8611, 0.8595, 0.8585, 0.8538, 0.8526, 0.8476, 0.8472, 0.8455, 0.8440], device='cuda:0')
data: tensor([[3.9252e+02, 1.0020e+03, 3.8274e+03, 1.1792e+03, 9.0741e-01, 0.0000e+00],
        [4.2550e+02, 1.9297e+03, 3.8401e+03, 2.1145e+03, 9.0562e-01, 0.0000e+00],
        [3.8332e+02, 1.4708e+03, 3.8090e+03, 1.6798e+03, 9.0331e-01, 0.0000e+

In [180]:
from PIL import Image

# 잘라내기 (이거 완성 후에는 모델 내에서 잘라내는 것으로)
V, H, c = results[0].orig_img.shape
xyxy = boxes.xyxy.cpu().numpy()
if xyxy.shape[0] != 36:
    pass
    #continue

# xyxy = np.concatenate([xyxy, np.array(cur_cls).reshape(-1,1)], axis=1)
sorted_data = xyxy[np.lexsort((xyxy[:,1], xyxy[:,0]))]

kit = sorted_data[:3]
sensor = sorted_data[3:]
kit = kit[np.lexsort((kit[:,0], kit[:,1]))]
groups = []

for k in kit:
    # k[0]: lower bound for x, k[2]: upper bound for x
    # k[1]: lower bound for y, k[3]: upper bound for y
    mask = (sensor[:, 0] > k[0]) & (sensor[:, 0] < k[2]) & \
           (sensor[:, 1] > k[1]) & (sensor[:, 1] < k[3])
    result = sensor[mask]
    result = result[np.lexsort((result[:,1], result[:,0]))]

    groups.append(result)

RESIZE_SIZE = 64          # 개별 이미지 크기 (64x64)
GRID_ROWS = 4             # 그리드 행 수
GRID_COLS = 3             # 그리드 열 수
COMPOSITE_SIZE_ROW = RESIZE_SIZE * GRID_ROWS  # 최종 대시보드 이미지 크기 
COMPOSITE_SIZE_COL = RESIZE_SIZE * GRID_COLS  # 최종 대시보드 이미지 크기 


# 각 그룹 출력
for idx, (kit, group) in enumerate(zip(kit, groups)):
    print(f"Kit {idx+1}")
    print(kit)
    print(f"Kit {idx+1}에 해당하는 sensor 데이터:")
    # print(group, "\n")
    # 6x6 그리드 composite 이미지 생성 (배경: 검은색)
    composite_img = Image.new("RGB", (COMPOSITE_SIZE_COL, COMPOSITE_SIZE_ROW), color=(0, 0, 0))
    # cur_kit_images = []
    for j, patch in enumerate(group):
        print(patch)
        x1, y1, x2, y2 = patch
        crop_patch = results[0].orig_img[round(y1):round(y2), round(x1):round(x2)]
        # crop_patch = cv2.cvtColor(crop_patch.astype(np.uint8), cv2.COLOR_BGR2RGB)

        # cur_kit_images.append(crop_patch)
    
        # for j, file_path in enumerate(composite_files):
        row = j // GRID_COLS
        col = j % GRID_COLS
        x_offset = col * RESIZE_SIZE
        y_offset = row * RESIZE_SIZE
        crop_patch = crop_patch[..., ::-1]
        # v1: 전체 이미지를 그대로 리사이징
        crop_patch_pil = Image.fromarray(crop_patch)
        crop_patch_pil = crop_patch_pil.resize((RESIZE_SIZE, RESIZE_SIZE))
    
        composite_img.paste(crop_patch_pil, (x_offset, y_offset))
        
    out_filename = f"dashboard_{idx:03d}.jpg"
    out_path = os.path.join(out_filename)
    composite_img.save(out_path)
    print(f"저장: {out_path}")


Kit 1
[     392.52        1002      3827.4      1179.2]
Kit 1에 해당하는 sensor 데이터:
[      474.1      1028.3      615.55      1154.8]
[     674.75      1026.9      815.38      1153.9]
[     880.42      1024.2      1021.9      1153.4]
[       1088      1022.5      1231.2      1152.9]
[     1295.5      1023.3      1436.6      1152.3]
[     1499.4      1025.7      1642.2      1156.7]
[     1710.6      1022.9      1852.7      1154.7]
[     1916.8      1021.3      2057.8      1151.4]
[     2122.4      1020.5      2265.4      1151.9]
[     2331.5      1023.2      2473.1      1152.8]
[     2534.6      1023.4      2676.7      1153.3]
저장: dashboard_000.jpg
Kit 2
[     383.32      1470.8        3809      1679.8]
Kit 2에 해당하는 sensor 데이터:
[     448.36      1539.9      591.27      1671.7]
[     652.65      1540.5      792.35      1668.2]
[     857.31      1537.9      996.44        1664]
[     1064.2      1533.7      1203.7      1660.3]
[     1270.9      1530.4      1408.3      1654.8]
[     1479.4      

In [187]:
import torchvision.transforms as T

image_size = 32

transform = T.Compose([
            T.Resize((image_size, image_size)),
            T.ToTensor(),  # Converts to (C, H, W)
        ])


# 각 그룹 출력
batch = []
for group in groups:
    print(f"cur Kit 에 해당하는 sensor 데이터:")
    images = []
    for j, patch in enumerate(group):
        print(patch)
        x1, y1, x2, y2 = patch
        crop_patch = results[0].orig_img[round(y1):round(y2), round(x1):round(x2)]
        crop_patch = crop_patch[..., ::-1]
        # v1: 전체 이미지를 그대로 리사이징
        crop_patch_pil = Image.fromarray(crop_patch)
        img = transform(crop_patch_pil)  # (3, 32, 32)
        images.append(img)
    images = torch.stack(images)
    batch.append(images)
batch = torch.stack(batch)

print(batch.shape)

cur Kit 에 해당하는 sensor 데이터:
[      474.1      1028.3      615.55      1154.8]
[     674.75      1026.9      815.38      1153.9]
[     880.42      1024.2      1021.9      1153.4]
[       1088      1022.5      1231.2      1152.9]
[     1295.5      1023.3      1436.6      1152.3]
[     1499.4      1025.7      1642.2      1156.7]
[     1710.6      1022.9      1852.7      1154.7]
[     1916.8      1021.3      2057.8      1151.4]
[     2122.4      1020.5      2265.4      1151.9]
[     2331.5      1023.2      2473.1      1152.8]
[     2534.6      1023.4      2676.7      1153.3]
cur Kit 에 해당하는 sensor 데이터:
[     448.36      1539.9      591.27      1671.7]
[     652.65      1540.5      792.35      1668.2]
[     857.31      1537.9      996.44        1664]
[     1064.2      1533.7      1203.7      1660.3]
[     1270.9      1530.4      1408.3      1654.8]
[     1479.4      1527.9      1614.4      1651.9]
[     1682.2      1523.1      1821.2      1647.5]
[     1891.8      1518.4      2025.3      1640

In [188]:
import torch
import torch.nn as nn

class ImageSetViT(nn.Module):
    def __init__(self, patch_dim=3*32*32, embed_dim=256, num_patches=11, num_heads=4, num_classes=5, depth=4):
        super().__init__()
        self.num_patches = num_patches
        self.patch_dim = patch_dim

        # Linear embedding: 각 이미지(=패치)를 벡터로
        self.embedding = nn.Linear(patch_dim, embed_dim)

        # Learnable positional encoding
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches, embed_dim))

        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=depth)

        # Classification head
        self.head = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim, num_classes)
        )

    def forward(self, x):
        # x: (B, 11, 3, 32, 32)
        B, N, C, H, W = x.shape
        x = x.view(B, N, -1)  # (B, 11, 3*32*32)
        x = self.embedding(x)  # (B, 11, embed_dim)
        x = x + self.pos_embedding  # positional encoding

        x = self.transformer(x)  # (B, 11, embed_dim)
        x = x.mean(dim=1)  # aggregate over patches

        out = self.head(x)  # (B, num_classes)
        return out


In [199]:
model = ImageSetViT()
model

ImageSetViT(
  (embedding): Linear(in_features=3072, out_features=256, bias=True)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (linear1): Linear(in_features=256, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=256, bias=True)
        (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (head): Sequential(
    (0): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (1): Linear(in_features=256, out_features=5, bias=True)
  )
)

In [200]:
device = "cuda"

model.to(device)
model.eval()
with torch.no_grad():
    x = batch[0:1].to(device)
    logits = model(x)
    probs = torch.sigmoid(logits)
    print(probs)
    preds = (probs > 0.5).float()

    print("예측 결과:", preds[0])
    



tensor([[0.4527, 0.4858, 0.5246, 0.4058, 0.3553]], device='cuda:0')
예측 결과: tensor([0., 0., 1., 0., 0.], device='cuda:0')


In [100]:
import torch
from torch import nn
import torch.nn.functional as F
from einops import rearrange, repeat
from einops.layers.torch import Rearrange

# helpers

def pair(t):
    return t if isinstance(t, tuple) else (t, t)

# classes

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head * heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.norm = nn.LayerNorm(dim)

        self.attend = nn.Softmax(dim = -1)
        self.dropout = nn.Dropout(dropout)

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        x = self.norm(x)
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale
        attn = self.attend(dots)
        attn = self.dropout(attn)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout),
                FeedForward(dim, mlp_dim, dropout = dropout)
            ]))

    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return self.norm(x)

class ViT(nn.Module):
    """
    수정된 ViT 모듈:
      - 이미지와 해당 이미지의 바운딩 박스(boxes)를 입력받습니다.
      - 각 바운딩 박스 영역을 crop한 후 patch_size로 resize하여 패치로 사용합니다.
      - 각 이미지에 대해, 바운딩 박스들을 y1 좌표(세로 기준)로 정렬하여 순서를 고정합니다.
    """
    def __init__(self, *, patch_size, num_boxes, num_classes, dim, depth, heads, mlp_dim,
                 pool='cls', channels=3, dim_head=64, dropout=0., emb_dropout=0.):
        super().__init__()
        patch_height, patch_width = pair(patch_size)
        patch_dim = channels * patch_height * patch_width  # flatten한 패치의 차원

        # 각 crop된 patch를 flatten하고 embedding하는 부분
        self.to_patch_embedding = nn.Sequential(
            Rearrange('b n c h w -> b n (c h w)'),
            nn.LayerNorm(patch_dim),
            nn.Linear(patch_dim, dim),
            nn.LayerNorm(dim),
        )

        # pos_embedding의 개수는 (num_boxes + 1) (cls token 포함)
        self.pos_embedding = nn.Parameter(torch.randn(1, num_boxes + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()
        self.mlp_head = nn.Linear(dim, num_classes)
        self.patch_size = (patch_height, patch_width)  # forward에서 resize할 크기

    def forward(self, img, boxes):
        """
        Args:
          img: 입력 이미지 텐서, shape (b, c, H, W)
          boxes: 바운딩 박스 텐서, shape (b, num_boxes, 4)
                 각 box는 (x1, y1, x2, y2) 순이며, 픽셀 좌표(정수)를 가정합니다.
        """
        b = img.shape[0]
        num_boxes = boxes.shape[1]
        patches = []

        for i in range(b):
            img_i = img[i]  # (c, H, W)

            # 각 이미지의 바운딩 박스들을 y1 좌표 기준으로 정렬 (vertical 순서 유지)
            boxes_i = boxes[i]  # (num_boxes, 4)
            print(boxes_i)
            # boxes_i[:, 1]가 y1 좌표입니다.
            sorted_indices = torch.argsort(boxes_i[:, 1])
            boxes_i = boxes_i[sorted_indices]

            patches_per_image = []
            for box in boxes_i:
                # (x1, y1, x2, y2) 좌표를 정수로 변환
                x1, y1, x2, y2 = [int(coord.item()) for coord in box]
                # 이미지 crop (채널 차원은 그대로 유지)
                patch = img_i[:, y1:y2, x1:x2]  # (c, h_box, w_box)
                # patch 크기가 patch_size와 다를 수 있으므로 resize
                patch = patch.unsqueeze(0)  # (1, c, h_box, w_box)
                patch = F.interpolate(patch, size=self.patch_size, mode='bilinear', align_corners=False)
                patch = patch.squeeze(0)  # (c, patch_height, patch_width)
                patches_per_image.append(patch)
            # (num_boxes, c, patch_height, patch_width)
            patches.append(torch.stack(patches_per_image, dim=0))
        # (b, num_boxes, c, patch_height, patch_width)
        patches = torch.stack(patches, dim=0)

        # patch embedding: flatten 각 패치를 벡터로 변환 후 embedding
        x = self.to_patch_embedding(patches)  # (b, num_boxes, dim)

        # cls token 추가
        cls_tokens = repeat(self.cls_token, '1 1 d -> b 1 d', b=b)
        x = torch.cat((cls_tokens, x), dim=1)  # (b, num_boxes + 1, dim)

        # 위치 임베딩 적용 (cls token 포함)
        x = x + self.pos_embedding[:, : (num_boxes + 1)]
        x = self.dropout(x)

        # Transformer 통과
        x = self.transformer(x)

        # pool 방식에 따라 최종 토큰 선택
        x = x.mean(dim=1) if self.pool == 'mean' else x[:, 0]
        x = self.to_latent(x)
        return self.mlp_head(x)


In [101]:
vitmodel = ViT(
    patch_size = 32,
    num_boxes = 36,
    num_classes = 2,
    dim = 1024,
    depth = 6,
    heads = 16,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1
)

In [102]:
vitmodel(results[0].orig_img, cur_xywhn)

['0.527491' '0.363526' '0.858719' '0.059048']


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed